## Collect newly deposited datasets, add them to the MongoDB instance 

### Load data

In [1]:
import sys
# Specify the path to the scripts folder
sys.path.append('../scripts/')
from mongodb import load_data

In [2]:
collection = load_data()

### Find latest date of deposit

In [ ]:
# Find the latest entry for the field "ore:describes.dateOfDeposit"
latest_entry = collection.find_one(
    {"ore:describes.dateOfDeposit": {"$exists": True}},
    sort=[("ore:describes.dateOfDeposit", -1)]
)
print(latest_entry)

In [13]:
latest_entry["ore:describes"]["dateOfDeposit"]

'2025-01-13'

### Extract all metadata for datasets after latest_entry 
Query `q_2025` selects all datasets deposited to the Archaeology Data Station between 2025-01-13 and 2025-03-28 (date of writing), with the Published status. 


In [73]:
q_2025 = "https://archaeology.datastations.nl/api/search?q=*&per_page=1000&type=dataset&show_facets=true&fq=publicationDate:2025&fq=dateSort:[2025-01-13T00:00:00Z+TO+2025-03-27T00:00:00Z]&fq=publicationStatus:Published"

In [74]:
q_2025

'https://archaeology.datastations.nl/api/search?q=*&per_page=1000&type=dataset&show_facets=true&fq=publicationDate:2025&fq=dateSort:[2025-01-13T00:00:00Z+TO+2025-03-27T00:00:00Z]&fq=publicationStatus:Published'

In [77]:
## Automate the process of fetching all the data from the API
dois = []

import urllib
import json
base = q_2025
rows = 1000
start = 0
page = 1
condition = True # emulate do-while
while (condition):
    url = base + "&start=" + str(start)
    data = json.load(urllib.request.urlopen(url))
    total = data['data']['total_count']
    print("=== Page", page, "===")
    print("start:", start, " total:", total)
    for i in data['data']['items']:
        #print("- ", i['name'], "(" + i['type'] + ")")
        doi = i['url']
        dois.append(doi)
    start = start + rows
    page += 1
    condition = start < total

=== Page 1 ===
start: 0  total: 3232
=== Page 2 ===
start: 1000  total: 3232
=== Page 3 ===
start: 2000  total: 3232
=== Page 4 ===
start: 3000  total: 3232


In [ ]:
# Write to .txt file 
# with open('../../data/dois_2025-03-28.txt', 'w') as f:
#     for item in dois:
#         f.write("%s\n" % item)